In [19]:
import os 

In [20]:
%pwd

'g:\\MLOps_Project\\MLOps_Project'

In [3]:
os.chdir("../")

In [21]:
%pwd

'g:\\MLOps_Project\\MLOps_Project'

In [22]:
import pandas as pd
data = pd.read_csv("artifacts/data_ingestion/2687_capstone_project_dataset_v1_vv6_ahjq7xz (1).csv",index_col=0)

In [23]:
data.head()
# data.drop(columns=["Unnamed: 0"],inplace=True)
data.head()

,YearsExperience,Salary
0,1.2,39344.0
1,1.4,46206.0
2,1.6,37732.0
3,2.1,43526.0
4,2.3,39892.0


In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30 entries, 0 to 29
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   YearsExperience  30 non-null     float64
 1   Salary           30 non-null     float64
dtypes: float64(2)
memory usage: 720.0 bytes


In [25]:
data.isnull().sum()

YearsExperience    0
Salary             0
dtype: int64

In [26]:
data.shape

(30, 2)

In [27]:
data.columns

Index(['YearsExperience', 'Salary'], dtype='object')

In [28]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [29]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [30]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [31]:
import os 
from mlProject import logger

In [39]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            # Load data
            data = pd.read_csv(self.config.unzip_data_dir, index_col=0)
            print("Data columns:", data.columns.tolist())

            # Schema columns
            all_schema = list(self.config.all_schema.keys())
            print("Schema columns:", all_schema)

            # Compare sets
            data_cols = set(data.columns)
            schema_cols = set(all_schema)

            if data_cols == schema_cols:
                validation_status = True
            else:
                missing_in_data = schema_cols - data_cols
                extra_in_data = data_cols - schema_cols
                print(f"❌ Missing in data: {missing_in_data}")
                print(f"❌ Extra in data: {extra_in_data}")
                validation_status = False

            # Save result once
            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation status: {validation_status}")

            return validation_status

        except Exception as e:
            raise e


Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1078, in _pydevd_bundle.pydevd_cython.PyDBFrame.trace_dispatch
  File "_pydevd_bundle/pydevd_cython.pyx", line 297, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "c:\Users\ADARSH\anaconda3\lib\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 1976, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
  File "c:\Users\ADARSH\anaconda3\lib\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 2011, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-09-06 17:28:27,781: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-06 17:28:27,785: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-06 17:28:27,789: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-09-06 17:28:27,792: INFO: common: created directory at: artifacts]
[2025-09-06 17:28:27,795: INFO: common: created directory at: artifacts/data_validation]
Index(['YearsExperience', 'Salary'], dtype='object')
['YearsExperience', 'Salary'] all_cols
a ['YearsExperience', 'Salary']
